In [1]:
import pydra
from pydra.tasks import fsl
from pathlib import Path
from ipyniivue import Niivue
import shutil
import pydra
import os
import nest_asyncio
import shutil
nest_asyncio.apply()
deface_state = False
from pathlib import Path
import os

import shutil
import nibabel as nib
import numpy as np

ModuleNotFoundError: No module named 'pydra'

In [ ]:
patientdir = Path("/Users/rosana.eljurdi/Datasets/Lymphoma")
labeldir = Path(patientdir, "derivatives/manual_segm")


label_paths = list(sorted(labeldir.rglob('*.nii.gz')))
image_paths = list(sorted(set(patientdir.rglob('*.nii.gz')) - set(label_paths)))

image_paths
print('hello')

In [3]:
def deface_and_resample_to_template(**kwargs) -> pydra.Workflow:
    from pydra.tasks import fsl
    from pydra.tasks.fsl import fslmaths

    workflow = pydra.Workflow(
        input_spec=[
            "input_image",
            "template_image",
            "template_mask",
            "segm_image"
        ],
        **kwargs,
    )

    workflow.add(
        fsl.FSLReorient2Std(
            name="fslreorient2std",
            input_image=workflow.lzin.input_image,
        )
    )

    reference_image = (
        workflow.lzin.template_image
    )

    workflow.add(
        fsl.FLIRT(
            name="flirt",
            input_image=workflow.fslreorient2std.lzout.output_image,
            reference_image=workflow.lzin.template_image,
            cost_function="mutualinfo",
        )
    )

    workflow.add(
        fslmaths.Mul(
            name="apply_mask",
            input_image=workflow.flirt.lzout.output_image,
            other_image=workflow.lzin.template_mask,
        )
    )
    
    workflow.add(
        fsl.ApplyXFM(
            name="apply_xfm",
            input_image=workflow.lzin.segm_image,
            reference_image=reference_image,
            input_matrix=workflow.flirt.lzout.output_matrix,
            interpolation="nearestneighbour",
        )
    )
    


    workflow.set_output({
        "output_image": workflow.apply_mask.lzout.output_image,
        "registered_GT": workflow.apply_xfm.lzout.output_image,
        "output_matric": workflow.flirt.lzout.output_matrix
    })

    return workflow


In [4]:
for image_path in image_paths:
    new_path = Path(str(image_path.parent).replace('Lymphoma', 'Lymphoma-defaced-brain-mni'))
    new_path.mkdir(parents=True, exist_ok=True)
    print(new_path)
    
    
for label_path in label_paths:
    new_path = Path(str(label_path.parent).replace('Lymphoma', 'Lymphoma-defaced-brain-mni'))
    new_path.mkdir(parents=True, exist_ok=True)
    
    print(new_path)

/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070077/ses-M000/anat
/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070222/ses-M000/anat
/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070224/ses-M000/anat
/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070230/ses-M000/anat
/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070336/ses-M000/anat
/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070535/ses-M000/anat
/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070568/ses-M000/anat
/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070570/ses-M000/anat
/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070629/ses-M000/anat
/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070741/ses-M000/anat
/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070898/ses-M000/anat
/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1071074/ses-M0

In [ ]:
# defacing 

for patient_path in enumerate(zip(image_paths, label_paths)):

    Task = deface_and_resample_to_template(input_image=patient_path[1][0], name='deface', 
              template_image='/Users/rosana.eljurdi/Datasets/defacing-templates/tpl-MNI152Lin_res-01_T1w.nii.gz', 
              template_mask = '/Users/rosana.eljurdi/Datasets/defacing-templates/tpl-MNI152Lin_res-01_desc-brain_mask.nii.gz',
              segm_image = patient_path[1][1]  ,
             )

    result = Task()
    
    
    shutil.copy(str(result.output.output_image), str(patient_path[1][0]).replace("Lymphoma", "Lymphoma-defaced-brain-mni"))
    shutil.copy(str(result.output.registered_GT), str(patient_path[1][1]).replace("Lymphoma", "Lymphoma-defaced-brain-mni"))
    print(str(patient_path).replace("Lymphoma", "Lymphoma-defaced-brain-mni"))
    print(result.output)
    print(patient_path)

(0, (PosixPath('/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070077/ses-M000/anat/sub-1070077_ses-M000_T1w.nii.gz'), PosixPath('/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/derivatives/manual_segm/sub-1070077/ses-M000/anat/sub-1070077_ses-M000_dseg.nii.gz')))
Output(output_image=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmpxhxxkcix/Workflow_ddc8189d695a1491238dd6cb1a63b6a68223d6cb7106498e8b50a82de46c3c21/sub-1070077_ses-M000_T1w_r2std_flirt_fslmaths.nii.gz'), registered_GT=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmpxhxxkcix/Workflow_ddc8189d695a1491238dd6cb1a63b6a68223d6cb7106498e8b50a82de46c3c21/sub-1070077_ses-M000_dseg_axfm.nii.gz'), output_matric=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmpxhxxkcix/Workflow_ddc8189d695a1491238dd6cb1a63b6a68223d6cb7106498e8b50a82de46c3c21/sub-1070077_ses-M000_T1w_r2std_flirt.mat'))
(0, (PosixPath('/Users/rosana.eljurdi/Datasets/Lymphoma/su

(7, (PosixPath('/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/sub-1070570/ses-M000/anat/sub-1070570_ses-M000_T1w.nii.gz'), PosixPath('/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/derivatives/manual_segm/sub-1070570/ses-M000/anat/sub-1070570_ses-M000_dseg.nii.gz')))
Output(output_image=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmpoex_lfqa/Workflow_5cb4cb325a35ddd74ff0eac8382c431e02d85d046d8a0903c2d1990ea2f80c59/sub-1070570_ses-M000_T1w_r2std_flirt_fslmaths.nii.gz'), registered_GT=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmpoex_lfqa/Workflow_5cb4cb325a35ddd74ff0eac8382c431e02d85d046d8a0903c2d1990ea2f80c59/sub-1070570_ses-M000_dseg_axfm.nii.gz'), output_matric=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmpoex_lfqa/Workflow_5cb4cb325a35ddd74ff0eac8382c431e02d85d046d8a0903c2d1990ea2f80c59/sub-1070570_ses-M000_T1w_r2std_flirt.mat'))
(7, (PosixPath('/Users/rosana.eljurdi/Datasets/Lymphoma/su

In [ ]:

# Sanity check: 

for image_path in enumerate(zip(image_paths, label_paths)):
        img = nib.load(image_path[1][0]).get_fdata()
        gt = nib.load(image_path[1][1]).get_fdata()
        print(os.path.basename(image_path[1][0]), img.shape,gt.shape )
        assert (img.shape == gt.shape)
   

In [ ]:

#sanity check 2
for patient_path in image_paths:
    
    task = fsl.FSLInfo(input_image=patient_path)
    result = task()
    
    print(result.output.dim1, result.output.dim2, result.output.dim3)